In [2]:
#general instructions

#2 or more patient encounters are required as the authors remove the last encounter while making predictions
# patient mortality is the outcome label and aquired from mimic. 
#2825 out of 7537 patients died (37.9%)
#Data set was divided into  train (5275/7537), validation (753/7537) and test (1509/7537)
# Once the optimal parameters were selected, the model was retrained by combining train and validation (6028/7537)
#
#
#********************** stars mean the data is not correct but the implemention functions.
#

In [3]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import time
import sys
import pickle
import math
import argparse
import torch.nn.functional as F
from torch.autograd import Variable
from tqdm import tqdm
from __future__ import division
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

In [4]:
# set seed - only really necessary to compare results with each other, but can remove from the submission
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# define data path
DATA_PATH = "~/Documents/DLH/mimic-iii-clinical-database-1.4"


# define data - separated to show which modules load (some take much longer than others)
#The commented out sections take a long time to load, and weren't immidiately useful at this stage
#Can uncommment when relevant

#TODO
# - reorganize into a definition() after  

In [5]:
Patients = pd.read_csv(DATA_PATH + '/PATIENTS.csv')

In [6]:
Diagnosis = pd.read_csv(DATA_PATH + '/DIAGNOSES_ICD.csv')

In [7]:
Procedures = pd.read_csv(DATA_PATH + '/D_ICD_PROCEDURES.csv')

In [8]:
#Prescriptions = pd.read_csv(DATA_PATH + '/PRESCRIPTIONS.csv')

In [9]:
LabEvents = pd.read_csv(DATA_PATH + '/LABEVENTS.csv')

In [10]:
LabLabels = pd.read_csv(DATA_PATH + '/D_LABITEMS.csv')

In [11]:
Admissions = pd.read_csv(DATA_PATH + '/ADMISSIONS.csv')

In [12]:
ProcedureLabels = pd.read_csv(DATA_PATH + '/D_ITEMS.csv')

In [13]:
IcuStays = pd.read_csv(DATA_PATH + '/ICUSTAYS.csv')

In [14]:
#ChartEvents = pd.read_csv(DATA_PATH + '/CHARTEVENTS.csv')

In [32]:
# patient inclusion - Only ICU patients with 2 or more visits are included. Patient IDs gathered
#May be incorrect as the paper specifies only 7,537 patients in their list.
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=sys.maxsize)

#********************************************
#Generate an ignore list for later ease of use.
#Manually verified that all the included patients have 2 or more subject ids. Refine by HADM_ID? ICUSTAY_ID?

def Patient_Inclusion(IcuStays):
    IcuList = IcuStays.to_numpy()

    unique, counts = np.unique(IcuList[1:, 1], return_counts=True)
    
    print("Max encouters should equal 42?", "max counts: ", max(counts))
    
    AllPatients = np.array(list(zip(np.transpose(unique), np.transpose(counts))))
    
    PatientList = []
    IgnoreList = []
    
    for i in AllPatients:
        if i[1] >= 2:
            PatientList.append(i)
        else:
            IgnoreList.append(i[0])
            
          
    #uniquehadm, counts2 = np.unique(IcuList[1:, 1], return_counts=True)
    
    
            
    #hadm_ID = IcuStays[['SUBJECT_ID', 'HADM_ID']]
    
    #IcuStay_ID = IcuStays[['SUBJECT_ID', 'ICUSTAY_ID']]
    
    #unique2= pd.DataFrame(PatientList, columns = ["SUBJECT_ID", "Count"])

    #print(len(PatientList))
    
    #HadmIcuList = hadm_ID[hadm_ID.SUBJECT_ID.isin(unique2.SUBJECT_ID)]
    #HadmIcuList = HadmIcuList.drop_duplicates("HADM_ID")
    #uniquehadm, counts2 = np.unique(IcuList[1:, 1], return_counts=True)
    
    #hadm2 = np.array(list(zip(np.transpose(uniquehadm), np.transpose(counts2))))
    #hadmlist, nothadm = [], []
    #for i in hadm2:
    #    if i[1] == 1:
    #        hadmlist.append(i)
    #    else:
    #        nothadm.append(i[0])
    
    
    #print(len(hadmlist))
    
    #print(hadmlist)
    
    
    
    #HadmIcu = HadmIcuList[HadmIcuList.SUBJECT_ID.isin(hadm_ID.SUBJECT_ID)]
    
    #print(HadmIcuList)
                

    #print(len(IgnoreList))
    return PatientList, IgnoreList

In [44]:
#used to create sequence lists like for Visit ID. 

def CreateList(n):
    lst = []
    for i in range(n):
        lst.append(i)
    return lst

In [63]:
#To be added to main at the end


#separate patient ids and count information
PatientList, IgnoreList = Patient_Inclusion(IcuStays)
PatientList = pd.DataFrame(PatientList, columns = ['ID', 'Count'])
Patient_ID = PatientList['ID']
Visits = PatientList['Count']
VisitID = [] * len(Visits)
count = 0
for i in Visits:
    temp = CreateList(i)
    VisitID.append(temp)
    count += 1


Max encouters should equal 42? max counts:  41


In [17]:
# data statistics - Ethnicity and visit statistics

def Ethnicity_Statistics(Admissions):
    white = 0
    black = 0
    hispanic = 0
    asian = 0
    other = 0
    
    #separate out ethnicity information
    EthStatistics = Admissions[['SUBJECT_ID', 'ETHNICITY']]
    IcuPatientEth = EthStatistics[EthStatistics.SUBJECT_ID.isin(Patient_ID)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)

    #Not yet working.... General idea.
    for i in IcuPatientEth.ETHNICITY:
        if "WHITE" in i:
            white += 1
        if "BLACK" in i:
            black += 1
        if "HISPANIC" in i:
            hispanic += 1
        if "ASIAN"  in i:
            asian += 1
    
        #I tried combining it, but the count was always more wrong...
        if "OTHER" in i:
            other += 1
        if "DECLINED" in i:
            other += 1
        if "NATIVE" in i:
            other += 1
        if "MIDDLE EASTERN" in i:
            other += 1
        if "MULTI" in i:
            other += 1
        if "UNABLE" in i:
            other += 1
        if "UNKNOWN" in i:
            other += 1
        

    print(white / len(IcuPatientEth), "percent white")
    print(black / len(IcuPatientEth), "percent black")
    print(hispanic / len(IcuPatientEth), "percent hispanic")
    print(asian / len(IcuPatientEth), "percent asian")
    print(other / len(IcuPatientEth), "percent other")

    print(((white + black + hispanic + asian + other) / len(IcuPatientEth)), "percent total")
    
    #*****************************************************
    #6 too many values. Need to sort that out.... maybe? it doesnt actually really affect anything...
    print(len(IcuPatientEth))
    print(white + black + asian + hispanic + other)
    return white, black, hispanic, asian, other

In [18]:
#To be added to main at the end
Ethnicity_Statistics(Admissions)

0.7356938892061679 percent white
0.10428326670474015 percent black
0.03438035408338092 percent hispanic
0.024785836664762993 percent asian
0.10154197601370646 percent other
1.0006853226727583 percent total
8755
8761


(6441, 913, 301, 217, 889)

In [19]:
#Age statistics - Date of birth DOB is in patients, and admissions time is in admissions.
# need to correlate both to find patients true ages
#Sorting to ensure that each patient id is matched in case of misordering
def Age_statistics(Patients):

    #separate out date of birth information and sort
    AgeStatistics = Patients[['SUBJECT_ID', 'DOB']]
    IcuPatientDob = AgeStatistics[AgeStatistics.SUBJECT_ID.isin(Patient_ID)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    IcuPatientDob = IcuPatientDob.sort_values(by=['SUBJECT_ID']).reset_index(drop = True)
    
    #separate out admission date and sort
    AdmStatistics = Admissions[['SUBJECT_ID', 'ADMITTIME']]
    IcuPatientAdm = AdmStatistics[AdmStatistics.SUBJECT_ID.isin(Patient_ID)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    IcuPatientAdm = IcuPatientAdm.sort_values(by=['SUBJECT_ID']).reset_index(drop = True)
    
    #minus dates to find ages
    #********************************************************
    # not done yet. need to convert at least part of the date to an int for each dataframe
    #Year and month should be sufficient, but year month date would increase accuracy. remove time.

    
    
Age_statistics(Patients)

In [20]:
#Sex statistics
def Gender_Statistics(Patients):
    male = 0
    female = 0
    
    SexStatistics = Patients[['SUBJECT_ID', 'GENDER']]
    IcuPatientSex = SexStatistics[SexStatistics.SUBJECT_ID.isin(Patient_ID)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    
    for i in IcuPatientSex.GENDER:
        if i == "M":
            male += 1
        if i == "F":
            female += 1
            
    print(male)
    print(female)
    
    print(male / len(IcuPatientSex), " percent male")
    print(female / len(IcuPatientSex), "percent female")
    
    return male, female

In [21]:
Gender_Statistics(Patients)

4943
3812
0.5645916619074814  percent male
0.43540833809251855 percent female


(4943, 3812)

In [83]:
#ICD Code statistics
def Icd_Codes(Diagnosis):
    
    #Isolate just the ICD codes and sequence of ICD codes
    IcdStatistics = Diagnosis[['SUBJECT_ID', 'ICD9_CODE', 'SEQ_NUM']]
    IcdStatistics.SEQ_NUM = IcdStatistics.SEQ_NUM.replace(np.nan,0).astype(int)
    IcuPatientIcd = IcdStatistics[IcdStatistics.SUBJECT_ID.isin(Patient_ID)].reset_index(drop = True)
    
    #For overall ICD Statistics
    EncounterTotal = IcuPatientIcd.drop_duplicates(['SUBJECT_ID', 'SEQ_NUM']).groupby('SUBJECT_ID')['SEQ_NUM'].count()
    EncounterMax = max(EncounterTotal)
    EncounterMean = np.mean(EncounterTotal)
    EncounterSTD = np.std(EncounterTotal)
    print(EncounterMax)
    print(EncounterMean)
    print(EncounterSTD)
    
    
    #For ICD code groupings by patient id
    IcdGroup = IcuPatientIcd.groupby("SUBJECT_ID")["SUBJECT_ID", "ICD9_CODE"]
    

    #for key, item in IcdGroup:
        #print(IcdGroup.get_group(key), "\n\n")
        #Data is ready but need to know the format before
        #*******************************************************
    
Icd_Codes(Diagnosis)

39
16.306453455168477
7.5563203100575596


/tmp/ipykernel_16821/1895454105.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IcdStatistics.SEQ_NUM = IcdStatistics.SEQ_NUM.replace(np.nan,0).astype(int)
/tmp/ipykernel_16821/1895454105.py:20: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  IcdGroup = IcuPatientIcd.groupby("SUBJECT_ID")["SUBJECT_ID", "ICD9_CODE"]


In [23]:
#Medication statistics
def Medications():
    #Need advice about medications for data science uses. In nursing, each medication is necessary 
    #however the authors mension removing duplicates per encounter. Should we only be keeping one medication
    #type per encounter? This doesnt seem like a good idea, but I am not sure if that is what the author is 
    #implying. Or, should we keep one medication type per day?
    pass

In [ ]:
#Lab component statistics
#The paper mentioned significant removal of duplicate patient information at this step (20% of data removed)
#a total of 5,609,021 lab tests were removed as they were not used.

def Laboratory(LabEvents):
    #Only include the abnormal flags, and ignore the actual values.
    LabStatistics = LabEvents[['SUBJECT_ID', 'ITEMID', 'FLAG']]
    IcuPatientLab = LabStatistics[LabStatistics.SUBJECT_ID.isin(Patient_ID)].reset_index(drop = True)
    
    #Separate the lab events into all found in ICU patients. not sure if needed
    IcuLabList = IcuPatientLab.drop_duplicates(['ITEMID'])
    
    #Gather only the abnormal flagged events
    IcuPatientLab = IcuPatientLab.dropna().reset_index(drop = True)

    #LabGroup = IcuPatientLab.groupby("SUBJECT_ID")["SUBJECT_ID", "ITEMID"]
    
    return IcuPatientLab
    
Laboratory(LabEvents)

In [ ]:
#Data loader


In [80]:
#Outcome label. Creates a list of 0's or 1's of length Patient_ID, where 1's indicate the patient passed away.


def Mortality(Admissions, Patient_ID):
    OutcomeFlag = Admissions[['SUBJECT_ID', 'HOSPITAL_EXPIRE_FLAG']]
    Outcome = OutcomeFlag[OutcomeFlag.SUBJECT_ID.isin(Patient_ID)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    Deceased = []


    for i in Outcome.to_numpy():
        if i[1] == 1:
            Deceased.append(i[0])
        
    Deceased = np.array(Deceased)
    Deceased = pd.DataFrame(Deceased, columns = ['Deceased']).to_numpy()

    DeceasedFlag = []
    for i in Patient_ID:
        if i in Deceased:
            DeceasedFlag.append(1)
        else:
            DeceasedFlag.append(0)
            
    return DeceasedFlag


In [ ]:
#Clout model (The most important model to finish)

#Source: https://github.com/subendhu19/CLOUT

In [ ]:
# Clout outline photos to work with



In [ ]:
# Auto Encoder (AE)

vocabsize_icd = 942
vocabsize_meds = 3202
vocabsize_labs = 284 #all 681 284

class AE(nn.Module):
    def __init__(self, epochs=5, batchsize=50, embsize=100):
        super(AE, self).__init__()
        self.epochs = epochs
        self.batchsize = batchsize
        self.embsize = embsize

        self.emb = nn.Linear(vocabsize_icd + vocabsize_meds + vocabsize_labs, self.embsize)

        self.out = nn.Linear(self.embsize, vocabsize_icd + vocabsize_meds + vocabsize_labs)

        self.reconloss = nn.MSELoss(size_average=True)

    def forward(self, input_icd, input_med, input_lab):

        input_full = torch.cat((input_icd, input_med, input_lab),1)

        hidden_full = F.relu(self.emb(input_full))	 

        output_full = F.relu(self.out(hidden_full))

        return [output_full, hidden_full]

    def get_encodings(self, ICD_data, Lab_data):
        return self.forward(Variable(torch.from_numpy(ICD_data).float()), Variable(torch.from_numpy(Lab_data).float()))[-1]

    def fit(self, ICDs, Meds, Labs):

        optimizer = optim.Adam(self.parameters(), 0.01)

        prev_loss = 1000
        for epoch in range(self.epochs):
            print 'Epoch:', epoch

            perm = np.random.permutation(ICDs.shape[0])
            ICDs = ICDs[perm]
            Meds = Meds[perm]
            Labs = Labs[perm]

            losses = []

            for i in range(0, ICDs.shape[0], self.batchsize):
                ICDbatch, Medbatch, Labbatch = ICDs[i:i+self.batchsize], Meds[i:i+self.batchsize], Labs[i:i+self.batchsize]
                ICDbatchvar, Medbatchvar, Labbatchvar = Variable(torch.from_numpy(ICDbatch).float()), Variable(torch.from_numpy(Medbatch).float()), Variable(torch.from_numpy(Labbatch).float())

                outputs = self.forward(ICDbatchvar, Medbatchvar, Labbatchvar)

                loss = self.reconloss(outputs[0], torch.cat((ICDbatchvar, Medbatchvar, Labbatchvar),1))

                losses.append(loss.data[0])

                optimizer.zero_grad()
               
                loss.backward()
                
                optimizer.step()
                # print 'recon loss:', loss_recon.data[0], 'loss_cr:', loss_cr.data[0]

            print 'Epoch loss:', np.mean(losses)

            if abs(np.mean(losses) - prev_loss) < 0.00005:
                break

            prev_loss = np.mean(losses)

model = AE(10,50,175)
ICD_data = pickle.load(open('../full data/CAE/CAEEntries.3digitICD9','r'))
Med_data = pickle.load(open('../full data/CAE/CAEEntries.meds','r'))
Lab_data = pickle.load(open('../full data/CAE/CAEEntries.abnlabs','r'))
model.fit(ICD_data, Med_data, Lab_data)

emb_weights = model._modules['emb'].weight.data.numpy().T
print 'Pickled embedding weights. Shape:', np.array(emb_weights).shape
pickle.dump(emb_weights, open('../full data/CAE/AE_embedding_weights.npy', 'wb'))

# print "Getting embeddings"
# outputs = []

# for i in tqdm(range(0, ICD_data.shape[0], 50)):
# 	ICDbatch, Labbatch = ICD_data[i:i+50], Lab_data[i:i+50]
# 	outputsbatch = model.get_encodings(ICDbatch, Labbatch).data.numpy()
# 	for ob in outputsbatch:
# 		outputs.append(ob)

# pickle.dump(np.array(outputs), open('../full data/CAE/AE_Embeddings', 'wb'))








In [ ]:
# Concatenate Auto Encoder (CAE)

vocabsize_icd = 942
vocabsize_meds = 3202
vocabsize_labs = 284 #all 681 284

class CAE(nn.Module):
    def __init__(self, epochs=5, batchsize=50, embsize=100, lamb=0.01):
        super(CAE, self).__init__()
        self.epochs = epochs
        self.batchsize = batchsize
        self.embsize = embsize
        self.lamb = lamb

        self.emb = nn.Linear(vocabsize_icd + vocabsize_meds + vocabsize_labs, self.embsize)

        self.out = nn.Linear(self.embsize, vocabsize_icd + vocabsize_meds + vocabsize_labs)

        self.reconloss = nn.MSELoss(size_average=True)

    def forward(self, input_icd, input_med, input_lab):

        input_full = torch.cat((input_icd, input_med, input_lab),1)
        input_onlyicd = torch.cat((input_icd, Variable(torch.zeros(input_med.size(0), input_med.size(1)).float()), Variable(torch.zeros(input_lab.size(0), input_lab.size(1)).float())), 1)
        input_onlymed = torch.cat((Variable(torch.zeros(input_icd.size(0), input_icd.size(1)).float()), input_med, Variable(torch.zeros(input_lab.size(0), input_lab.size(1)).float())), 1)
        input_onlylab = torch.cat((Variable(torch.zeros(input_icd.size(0), input_icd.size(1)).float()), Variable(torch.zeros(input_med.size(0), input_med.size(1)).float()), input_lab), 1)

        hidden_full = F.relu(self.emb(input_full))
        hidden_onlyicd = F.relu(self.emb(input_onlyicd))
        hidden_onlymed = F.relu(self.emb(input_onlymed))	
        hidden_onlylab = F.relu(self.emb(input_onlylab))		 

        output_full = F.relu(self.out(hidden_full))
        output_onlyicd = F.relu(self.out(hidden_onlyicd))
        output_onlymed = F.relu(self.out(hidden_onlymed))
        output_onlylab = F.relu(self.out(hidden_onlylab))	

        return [output_full, output_onlyicd, output_onlymed, output_onlylab, hidden_onlyicd, hidden_onlymed, hidden_onlylab, hidden_full]

    def get_encodings(self, ICD_data, Med_data, Lab_data):
        return self.forward(Variable(torch.from_numpy(ICD_data).float()), Variable(torch.from_numpy(Med_data).float()), Variable(torch.from_numpy(Lab_data).float()))[-1]

    def correlation_coef(self, x, y):
        vx = x - torch.mean(x)
        vy = y - torch.mean(y)

        cost = torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))
        return cost

    def joint_cumulant_by_var(self, x, y, z):	
        vx = x - torch.mean(x)
        vy = y - torch.mean(y)
        vz = z - torch.mean(z)

        cost = torch.sum(vx * vy * vz) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)) * torch.sqrt(torch.sum(vz ** 2)))
        return cost

        # e_xyz = torch.mean(x * y * z)
        # e_xy = torch.mean(x * y)
        # e_yz = torch.mean(y * z)
        # e_xz = torch.mean(x * z)
        # e_x = torch.mean(x)
        # e_y = torch.mean(y)
        # e_z = torch.mean(z)

        # kappa = e_xyz - (e_xy * e_z) - (e_xz * e_y) - (e_yz * e_x) + (2*e_x*e_y*e_z) 


    def fit(self, ICDs, Meds, Labs):

        optimizer = optim.Adam(self.parameters(), 0.01)

        prev_loss = 1000
        for epoch in range(self.epochs):
            print 'Epoch:', epoch

            perm = np.random.permutation(ICDs.shape[0])
            ICDs = ICDs[perm]
            Meds = Meds[perm]
            Labs = Labs[perm]

            losses = []

            for i in range(0, ICDs.shape[0], self.batchsize):
                ICDbatch, Medbatch, Labbatch = ICDs[i:i+self.batchsize], Meds[i:i+self.batchsize], Labs[i:i+self.batchsize]
                ICDbatchvar, Medbatchvar, Labbatchvar = Variable(torch.from_numpy(ICDbatch).float()), Variable(torch.from_numpy(Medbatch).float()), Variable(torch.from_numpy(Labbatch).float())

                outputs = self.forward(ICDbatchvar, Medbatchvar, Labbatchvar)

                loss_recon = self.reconloss(outputs[0], torch.cat((ICDbatchvar, Medbatchvar, Labbatchvar),1)) + self.reconloss(outputs[1], torch.cat((ICDbatchvar, Medbatchvar, Labbatchvar),1)) \
                        + self.reconloss(outputs[2], torch.cat((ICDbatchvar, Medbatchvar, Labbatchvar),1)) + self.reconloss(outputs[3], torch.cat((ICDbatchvar, Medbatchvar, Labbatchvar),1))

                loss_cr = self.joint_cumulant_by_var(outputs[4], outputs[5], outputs[6])

                loss = loss_recon - (self.lamb*loss_cr)

                losses.append(loss.data[0])

                optimizer.zero_grad()
                    
                loss.backward()
                
                optimizer.step()
                # print 'recon loss:', loss_recon.data[0], 'loss_cr:', loss_cr.data[0]

            print 'Epoch loss:', np.mean(losses)

            if abs(np.mean(losses) - prev_loss) < 0.00005:
                break

            prev_loss = np.mean(losses)


model = CAE(10,50,175,0.01)
ICD_data = pickle.load(open('../full data/CAE/CAEEntries.3digitICD9','r'))
Med_data = pickle.load(open('../full data/CAE/CAEEntries.meds','r'))
Lab_data = pickle.load(open('../full data/CAE/CAEEntries.abnlabs','r'))
model.fit(ICD_data, Med_data, Lab_data)

emb_weights = model._modules['emb'].weight.data.numpy().T
print 'Pickled embedding weights. Shape:', np.array(emb_weights).shape
pickle.dump(emb_weights, open('../full data/CAE/CAE_embedding_weights.npy', 'wb'))

# outputs = model.get_encodings(ICD_data, Med_data, Lab_data)
# print np.array(outputs).shape
# pickle.dump(outputs, open('../full data/CAE/Embeddings', 'wb'))









In [ ]:
#Clout model

class RNN(nn.Module):
    def __init__(self, epochs=5, batchsize=50, vocabsize=5, embsize=100):
        super(RNN, self).__init__()
        self.epochs = 5
        self.batchsize = batchsize
        self.vocabsize = vocabsize
        self.embsize = embsize

        self.emb_icd = nn.Linear(vocabsize_icd, embsize_icd)
        self.emb_meds = nn.Linear(vocabsize_meds, embsize_meds)
        self.emb_labs = nn.Linear(vocabsize_labs, embsize_labs)

        self.rnn = nn.LSTM(input_size=embsize, hidden_size=embsize, num_layers=1)
        self.out = nn.Linear(embsize, 1)
        self.sig = nn.Sigmoid()

    def forward(self, input_icd, input_med, input_lab, input_latent, hidden=None, force=True, steps=0):
        if force or steps == 0: steps = len(input_icd)
        outputs = Variable(torch.zeros(steps, 1, 1))

        input_icd = self.emb_icd(input_icd)
        input_med = self.emb_meds(input_med)
        input_lab = self.emb_labs(input_lab)

        inputs = F.relu(torch.cat((input_icd, input_med, input_lab, input_latent),1))

        inputs = inputs.view(inputs.size()[0],1,inputs.size()[1])
        outputs, hidden = self.rnn(inputs, hidden)
        outputs = self.out(outputs)
        return outputs.squeeze(), hidden

    def predict(self, input_icd, input_med, input_lab, input_latent):
        out, hid = self.forward(input_icd, input_med, input_lab, input_latent, None)
        return self.sig(out[-1]).data[0]

parser = argparse.ArgumentParser(description='Coq tactic prediction')
parser.add_argument('--emb_weights', type=str, default='../full data/CAE/CAE_embedding_weights.npy',
                    help='location of the embedding weight file (default: ../full data/CAE/CAE_embedding_weights.npy)')
args = parser.parse_args()

n_epochs = 1
vocabsize_icd = 942
vocabsize_meds = 3202
vocabsize_labs = 284 #all 681
vocabsize = vocabsize_icd+vocabsize_meds+vocabsize_labs

embsize_icd = 50
embsize_meds = 75
embsize_labs = 50
embsize_latent = 175
embsize = embsize_icd + embsize_labs + embsize_meds + embsize_latent

input_seqs_icd = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.3digitICD9.seqs')))
input_seqs_meds = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.meds.seqs')))
input_seqs_labs = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.abnlabs.seqs')))
# input_seqs_latent = np.array(pickle.load(open('../full data/CAE/Embeddings.seqs')))
latent_weights = pickle.load(open(args.emb_weights))

input_seqs_fullicd = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.seqs')))

icditems = pickle.load(open('../full data/type dictionaries/MIMICIIIPROCESSED.types', 'rb'))
meditems = pickle.load(open('../full data/type dictionaries/MIMICIIIPROCESSED.meds.types', 'rb'))
labitems = pickle.load(open('../full data/type dictionaries/MIMICIIIPROCESSED.abnlabs.types', 'rb'))

model_name = 'AE'
if args.emb_weights == '../full data/CAE/CAE_embedding_weights.npy':
    model_name = 'CAE'

interpretation_file = open("RNN_CLatent_Interpretations_" + model_name + ".txt", 'w')

# overall_risk_factor_file = open("risk_factors_averaged.txt", "w")

labnames = {}
lab_dict_file = open('D_LABITEMS.csv', 'r')
lab_dict_file.readline()
for line in lab_dict_file:
    tokens = line.strip().split(',')
    labnames[tokens[1].replace('"','')] = tokens[2]
lab_dict_file.close()

icdnames = {}
icd_dict_file = open('D_ICD_DIAGNOSES.csv', 'r')
icd_dict_file.readline()
for line in icd_dict_file:
    tokens = line.strip().split(',')
    icdnames[tokens[1].replace('"','')] = tokens[2]
icd_dict_file.close()

icd_scores = {}
med_scores = {}
lab_scores = {}

icd_totals = {}
med_totals = {}
lab_totals = {}

def get_ICD(icd):
    ret_str = ""
    icd_str = icditems.keys()[icditems.values().index(icd)]
    actual_key = icd_str.replace(".", "")[2:]
    if actual_key in icdnames:
        ret_str = icdnames[actual_key]
    else:
        ret_str = icd_str
    return ret_str

def get_med(med):
    ret_str = meditems.keys()[meditems.values().index(med)]
    return ret_str

def get_lab(lab):
    ret_str = labnames[labitems.keys()[labitems.values().index(lab)]]
    return ret_str

print 'Data loaded..'

labels = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.morts')))

trainratio = 0.7
validratio = 0.1
testratio = 0.2

trainlindex = int(len(input_seqs_icd)*trainratio)
validlindex = int(len(input_seqs_icd)*(trainratio + validratio))

print 'Data prepared..'

def convert_to_one_hot(code_seqs, len):
    new_code_seqs = []
    for code_seq in code_seqs:
        one_hot_vec = np.zeros(len)
        for code in code_seq:
            one_hot_vec[code] = 1
        new_code_seqs.append(one_hot_vec)
    return np.array(new_code_seqs)

def get_avg(seqs, type):
    count = 0
    for seq in seqs:
        count += len(seq)
    val = round(count*1.0/len(seqs))
    if type == 'i':
        return min(4, int(val/5))
    else:
        return min(4, int(val/50))

def get_factors(icd_seq, med_seq, lab_seq, model, actual_score, full_icd):
    potential_test_data = []

    for seq in range(len(icd_seq)):
        for i in range(len(icd_seq[seq])):
            potential_test_data.append(("icd", full_icd[seq][i], seq, icd_seq[:seq]+[icd_seq[seq][:i] + icd_seq[seq][i+1:]]+icd_seq[seq+1:], med_seq, lab_seq))
    for seq in range(len(med_seq)):
        for i in range(len(med_seq[seq])):
            potential_test_data.append(("med", med_seq[seq][i], seq, icd_seq, med_seq[:seq]+[med_seq[seq][:i]+med_seq[seq][i+1:]]+med_seq[seq+1:], lab_seq))
    for seq in range(len(lab_seq)):
        for i in range(len(lab_seq[seq])):
            potential_test_data.append(("lab", lab_seq[seq][i], seq, icd_seq, med_seq, lab_seq[:seq]+[lab_seq[seq][:i] + lab_seq[seq][i+1:]]+lab_seq[seq+1:]))

    risk_scores = []

    for pt in potential_test_data:
        test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(pt[3], vocabsize_icd)).float())
        test_input_med = Variable(torch.from_numpy(convert_to_one_hot(pt[4], vocabsize_meds)).float())
        test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(pt[5], vocabsize_labs)).float())

        latent_inputs_oh = np.concatenate((convert_to_one_hot(pt[3], vocabsize_icd), convert_to_one_hot(pt[4], vocabsize_meds), convert_to_one_hot(pt[5], vocabsize_labs)), 1)
        latent_inputs = np.dot(latent_inputs_oh, latent_weights)
        latent_inputs = Variable(torch.from_numpy(latent_inputs).float())

        factor_score = actual_score - model.predict(test_input_icd, test_input_med, test_input_lab, latent_inputs)
        factor = ""
        if pt[0] == 'icd':
            icd_tag = get_ICD(pt[1])
            factor = "ICD-"+icd_tag
            if icd_tag in icd_scores:
                icd_scores[icd_tag] += factor_score
                icd_totals[icd_tag] += 1
            else:
                icd_scores[icd_tag] = factor_score
                icd_totals[icd_tag] = 1
        elif pt[0] == 'med':
            med_tag = get_med(pt[1])
            factor = "Med-"+med_tag
            if med_tag in med_scores:
                med_scores[med_tag] += factor_score
                med_totals[med_tag] += 1
            else:
                med_scores[med_tag] = factor_score
                med_totals[med_tag] = 1
        else:
            lab_tag = get_lab(pt[1])
            factor = "Lab-"+lab_tag
            if lab_tag in lab_scores:
                lab_scores[lab_tag] += factor_score
                lab_totals[lab_tag] += 1
            else:
                lab_scores[lab_tag] = factor_score
                lab_totals[lab_tag] = 1
        risk_scores.append(("Encounter-"+str(pt[2])+": "+factor, factor_score))

    risk_scores.sort(key=lambda tup: tup[1], reverse=True)

    return risk_scores[:10]

print 'Starting training..'

batchsize = 50

# ICD_wise_tot_tr = np.zeros(5)
# meds_wise_tot_tr = np.zeros(5)
# labs_wise_tot_tr = np.zeros(5)

# for i in range(len(train_input_seqs_icd)):
# 	ICD_wise_tot_tr[get_avg(train_input_seqs_icd[i], 'i')] += 1
# 	meds_wise_tot_tr[get_avg(train_input_seqs_meds[i], 'm')] += 1
# 	labs_wise_tot_tr[get_avg(train_input_seqs_labs[i], 'l')] += 1

# print 'ICD-wise train total', ICD_wise_tot_tr
# print 'Meds-wise train total', meds_wise_tot_tr
# print 'Labs-wise train total', labs_wise_tot_tr

best_aucrocs = []
for run in range(10):
    print 'Run', run

    perm = np.random.permutation(input_seqs_icd.shape[0])
    rinput_seqs_icd = input_seqs_icd#[perm]
    rinput_seqs_meds = input_seqs_meds#[perm]
    rinput_seqs_labs = input_seqs_labs#[perm]
    # rinput_seqs_latent = input_seqs_latent[perm]
    rinput_seqs_fullicd = input_seqs_fullicd#[perm]
    rlabels = labels#[perm]

    train_input_seqs_icd = rinput_seqs_icd[:trainlindex]
    train_input_seqs_meds = rinput_seqs_meds[:trainlindex]
    train_input_seqs_labs = rinput_seqs_labs[:trainlindex]
    # train_input_seqs_latent = rinput_seqs_latent[:trainlindex]
    train_labels = rlabels[:trainlindex]
    train_labels = train_labels.reshape(train_labels.shape[0],1)

    valid_input_seqs_icd = rinput_seqs_icd[trainlindex:validlindex]
    valid_input_seqs_meds = rinput_seqs_meds[trainlindex:validlindex]
    valid_input_seqs_labs = rinput_seqs_labs[trainlindex:validlindex]
    # valid_input_seqs_latent = rinput_seqs_latent[trainlindex:validlindex]
    valid_labels = rlabels[trainlindex:validlindex]

    test_input_seqs_icd = rinput_seqs_icd[validlindex:]
    test_input_seqs_meds = rinput_seqs_meds[validlindex:]
    test_input_seqs_labs = rinput_seqs_labs[validlindex:]
    # test_input_seqs_latent = rinput_seqs_latent[validlindex:]
    test_input_seqs_fullicd = rinput_seqs_fullicd[validlindex:]
    test_labels = rlabels[validlindex:]

    n_iters = train_input_seqs_icd.shape[0]

    model = RNN(n_epochs, 1, vocabsize, embsize)
    criterion = nn.BCEWithLogitsLoss(size_average=False)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    aucrocs = []

    for epoch in range(n_epochs):
        epoch_loss = 0
        print 'Epoch', (epoch+1)

        for i in (range(0, n_iters, batchsize)):
            batch_icd = train_input_seqs_icd[i:i+batchsize]
            batch_meds = train_input_seqs_meds[i:i+batchsize]
            batch_labs = train_input_seqs_labs[i:i+batchsize]
            # batch_latent = train_input_seqs_latent[i:i+batchsize]

            batch_train_labels = train_labels[i:i+batchsize]

            optimizer.zero_grad()
            losses = []

            for i in range(len(batch_icd)):
                icd_onehot = convert_to_one_hot(batch_icd[i], vocabsize_icd)
                med_onehot = convert_to_one_hot(batch_meds[i], vocabsize_meds)
                lab_onehot = convert_to_one_hot(batch_labs[i], vocabsize_labs)

                icd_inputs = Variable(torch.from_numpy(icd_onehot).float())
                med_inputs = Variable(torch.from_numpy(med_onehot).float())
                lab_inputs = Variable(torch.from_numpy(lab_onehot).float())

                latent_inputs_oh = np.concatenate((icd_onehot, med_onehot, lab_onehot), 1)
                latent_inputs = np.dot(latent_inputs_oh, latent_weights)
                latent_inputs = Variable(torch.from_numpy(latent_inputs).float())

                # latent_inputs = Variable(torch.from_numpy(np.array(batch_latent[iter])).float())

                targets = Variable(torch.from_numpy(batch_train_labels[i]).float())

                # Use teacher forcing 50% of the time
                force = random.random() < 0.5
                outputs, hidden = model(icd_inputs, med_inputs, lab_inputs, latent_inputs, None, force)

                #print outputs[-1], targets
                losses.append(criterion(outputs[-1], targets))

            loss = sum(losses)/len(batch_icd)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.data[0]

        #print(epoch, epoch_loss)

        ## Validation phase
        vpredictions = np.zeros(len(valid_input_seqs_icd))
        for i in range(len(valid_input_seqs_icd)):
            test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(valid_input_seqs_icd[i], vocabsize_icd)).float())
            test_input_med = Variable(torch.from_numpy(convert_to_one_hot(valid_input_seqs_meds[i], vocabsize_meds)).float())
            test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(valid_input_seqs_labs[i], vocabsize_labs)).float())

            test_input_latent_oh = np.concatenate((convert_to_one_hot(valid_input_seqs_icd[i], vocabsize_icd), convert_to_one_hot(valid_input_seqs_meds[i], vocabsize_meds), convert_to_one_hot(valid_input_seqs_labs[i], vocabsize_labs)), 1)
            test_input_latent = np.dot(test_input_latent_oh, latent_weights)
            test_input_latent = Variable(torch.from_numpy(test_input_latent).float())

            # test_input_latent = Variable(torch.from_numpy(np.array(valid_input_seqs_latent[i])).float())
            vpredictions[i] = model.predict(test_input_icd, test_input_med, test_input_lab, test_input_latent)

        print "Validation AUC_ROC: ", roc_auc_score(valid_labels, vpredictions)

        ## Testing phase
        predictions = np.zeros(len(test_input_seqs_icd))

        # ICD_wise_corr = np.zeros(5)
        # meds_wise_corr = np.zeros(5)
        # labs_wise_corr = np.zeros(5)
        # ICD_wise_tot = np.zeros(5)
        # meds_wise_tot = np.zeros(5)
        # labs_wise_tot = np.zeros(5)

        for i in range(len(test_input_seqs_icd)):
            test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd)).float())
            test_input_med = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds)).float())
            test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)).float())

            test_input_latent_oh = np.concatenate((convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd), convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds), convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)), 1)
            test_input_latent = np.dot(test_input_latent_oh, latent_weights)
            test_input_latent = Variable(torch.from_numpy(test_input_latent).float())

            # test_input_latent = Variable(torch.from_numpy(np.array(test_input_seqs_latent[i])).float())
            predictions[i] = model.predict(test_input_icd, test_input_med, test_input_lab, test_input_latent)
            
            # ICD_wise_corr[get_avg(test_input_seqs_icd[i], 'i')] += int((predictions[i]>0.5)*1 == test_labels[i])
            # ICD_wise_tot[get_avg(test_input_seqs_icd[i], 'i')] += 1

            # meds_wise_corr[get_avg(test_input_seqs_meds[i], 'm')] += int((predictions[i]>0.5)*1 == test_labels[i])
            # meds_wise_tot[get_avg(test_input_seqs_meds[i], 'm')] += 1

            # labs_wise_corr[get_avg(test_input_seqs_labs[i], 'l')] += int((predictions[i]>0.5)*1 == test_labels[i])
            # labs_wise_tot[get_avg(test_input_seqs_labs[i], 'l')] += 1

        print "Test AUC_ROC: ", roc_auc_score(test_labels, predictions)

        aucrocs.append(roc_auc_score(test_labels, predictions))
        #fpr, tpr, _ = roc_curve(test_labels, predictions)
        #pickle.dump({"FPR":fpr, "TPR":tpr}, open('roc_clout_cornn.p', 'wb'))
        #actual_predictions = (predictions>0.5)*1
        #print classification_report(test_labels, actual_predictions)

    best_aucrocs.append(max(aucrocs))

print "Average AUCROC:", np.mean(best_aucrocs), "+/-", np.std(best_aucrocs) 

# print "Final testing and interpretations"
# predictions = np.zeros(len(test_input_seqs_icd))
# for i in (range(len(test_input_seqs_icd))):
# 	test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd)).float())
# 	test_input_med = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds)).float())
# 	test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)).float())
# 	test_input_latent_oh = np.concatenate((convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd), convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds), convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)), 1)
# 	test_input_latent = np.dot(test_input_latent_oh, latent_weights)
# 	test_input_latent = Variable(torch.from_numpy(test_input_latent).float())

# 	test_score = model.predict(test_input_icd, test_input_med, test_input_lab, test_input_latent)
# 	predictions[i] = test_score
# 	top_risk_factors = get_factors(test_input_seqs_icd[i], test_input_seqs_meds[i], test_input_seqs_labs[i], model, test_score, test_input_seqs_fullicd[i]) 
# 	if (test_score>0.5):
# 		interpretation_file.write("ID: " + str(i) + " True label: "+str(test_labels[i])+"\n")
# 		for rf in top_risk_factors:
# 			interpretation_file.write(str(rf)+"\n")
# 		interpretation_file.write("\n")

interpretation_file.close()

In [ ]:
#Clout training


In [ ]:
#Clout evaluation


In [ ]:
#Summary conclusions


In [ ]:
#Summary Graphs etc...
